<a href="https://colab.research.google.com/github/JonAnderDiez/ExpediaScraper/blob/master/CAF_PDF_SCRAPER_PRODUCTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONTEXTO

Este scraper pretende eliminar el trabajo repetitivo de extraer líneas, bobinas, mandos, pilotos y entradas/salidas de TCMS de los esquemas que solemos utilizar en el departamento de Material para simular los trenes más nuevos (generalmente de CAF).

**ACTUALMENTE YA SE PUEDE USAR** para extraer líneas que es lo más tedioso, pero dado que ya está todo preparado, resulta inmediato extraer las bobinas, pilotos, mandos, y I/Os de TCMS.

Como una imagen es más útil que mil palabras:

![](https://drive.google.com/uc?export=view&id=100BYZPQVQ3mzYKP74faBguiMTj6jvooj)

# TO-DO LIST

<font color='green'>DONE</font> VS. <font color='orange'>WORKING</font> VS. <font color='red'>TODO</font>

- <font color='green'>Evitar que los desarrolladores tengan que descargar Python para usar el SCRAPER -> [Google Colab](https://colab.research.google.com/notebooks/welcome.ipynb?hl=es) </font>

- <font color='green'> Eliminar la protección que CAF pone a los PDFs para tener derechos de escritura usando un método que no ponga en peligro la confidencialidad (open source, ejecutable en local y sin necesidad de conexión a ningún servidor) -> [Tutorial de uso](#EnlaceTutorialQuitarProteccion) </font>


- <font color='red'>   Quitar la protección a los PDFs de CAF dentro del scrip sin tener que usar aplicaciones de terceros. ->  [TODO](#)</font>

- <font color='green'> Extraer números de todas las **LÍNEAS** de cada esquema (se ha decidido realizar extracción por esquema y no por hoja para facilitar uso) -> [ScriptScraper](#EnlaceScriptExtraccionLineas) </font>

- <font color='orange'> Extraer nombre de **ELEMENTOS LISTADOS** (bobinas, mandos, diodos LED ...) de cada esquema (se ha decidido realizar extracción por esquema y no por hoja por facilitar uso) -> [ScriptScraper](#EnlaceScriptExtraccionLineas) </font>

- <font color='red'>Hacer que el TrainScription lea el CSV generado con el script y que cree automáticamente los métodos de todas las líneas y todas las entradas. (Ej.: Si estamos en el esquema 111 y tenemos una línea 33111, es evidente que la línea es una entrada al esquema y por lo tanto se podría programar la "PushEntrada" de dicha línea mediante RegEx). ->  [TODO](#)</font>

**NOTA:** Una vez se empiece a usar, podremos comprobar el número de fallos cometidos, si el número es significativo, para hacer la aplicación más robusta se podría probar con más librerías de extracción de texto  y/o cambiando el método de quitarle la protección a los PDFs de CAF.

<a name="EnlaceTutorialQuitarProteccion"></a>
# TUTORIAL DE USO

1. **PDF->WORD**: Converitr PDF descargado desde VISPLAN a Word usando un programilla que he encontrado en ''C#'' : [PDF->DOCX](https://drive.google.com/file/d/1JIgF9N5pcZZo15e7hI_ePwZYxVFO5uCe/view?usp=sharing)

![](https://drive.google.com/uc?export=view&id=1epXK4HEL1bivUiHJVU72YgA225jcOk9y)

**NOTA IMPORTANTE**: Es posible que la aplicación por unos segundos ponga que '(No responde)', pero siempre termina devolviendo un muy buen Word sin perder información por el camino.

---

2. **WORD->PDF**: En esta ocasión vamos a usar el propio Microsoft Word (o Libre Office, aunque no he probado con Libre Office). Hay que convertir el Word a PDF con un poco de cuidado para que puedan seguir leyéndose los elementos, y que no se conviertan en imágenes todas las páginas del PDF.

![](https://drive.google.com/uc?export=view&id=1LvhIz0cmDe73IA2KmwBaWNoXmum6ktDk)

---

3. **Guargar en Drive**: Hay que subir los PDFs resultantes a la carpeta de drive en el que se encuentre este Jupyter Notebook, copiar el código del PDF asignado en Drive, y pegarlo en el db.csv junto al nombre del PDF. Se generará un CSV por cada fila (PDF) de este db.csv.

![](https://drive.google.com/uc?export=view&id=1K6gW0LHdA7JK6Xyd7YVb-sMrdAKQfiur)


![](https://drive.google.com/uc?export=view&id=17C8ZrBZdh3qXDjYFhNiPvRSBziSiRJ2N)


![](https://drive.google.com/uc?export=view&id=1oJpA4N8DiCCvlaML_9lWZH33VX3X3aZD)

---

4. **Ejecutar script**: Dado que el script lee de Drive, hay que darle permisos de la siguiente forma:

![](https://drive.google.com/uc?export=view&id=1ZOaNNkLdchsufTNokf0C6ji2mEX4WAH5)

![](https://drive.google.com/uc?export=view&id=1w29XKb8NwMOHd35Ua5sl3wmY0OKFYUJu)

![](https://drive.google.com/uc?export=view&id=1g8-8GKn2Z0Gs10Q6WtoI3XwvF3Dmf0Qn)

![](https://drive.google.com/uc?export=view&id=1GaI0mm3PA6nqlUiXkowUoldEHUil67EV)

![](https://drive.google.com/uc?export=view&id=1VHRORdeQSyIgUysnoLyfGJ8o82puE1iU)

<a name="EnlaceScriptExtraccionLineas"></a>
# SCRIPT A EJECUTAR


In [ ]:
!pip install pandas
!pip install PyPDF2

import pandas as pd
import PyPDF2
import numpy as np
import re
import math

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

# 0 - PASAR DE STRING A LISTA DE STRINGS
def str2lst(string): 
    li = list(string.split("\n"))
    return li

# 1 - CREAR UN OBJETO DE PDF LEYENDO ARCHIVO EN DRIVE
def GetPDFObjFromDrive(CodigoDrive, NombrePDF):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  downloadedM = drive.CreateFile({'id': CodigoDrive})
  downloadedM.GetContentFile(NombrePDF)
  pdfFileObjM = open(NombrePDF, 'rb')
  pdfReader = PyPDF2.PdfFileReader(pdfFileObjM)
  return pdfReader

# 2 - CREAR DATAFRAME CON DIMENSIONES 'ESQUEMA' Y 'HOJA'
def GetDfEsquemaPorHoja(pdfReader):
  ## Sacar el código de la carpeta en la que guarda CAF los esquemas eléctricos del PDF actual.
  page = pdfReader.getPage(0)
  page.extractText()
  lista = (str2lst(page.extractText()))
  df1DTexto = pd.Series(lista)
  StringDeTextoLimpioDeEsquema = (df1DTexto[(df1DTexto.str.contains('\w.\w\w.\w\w.\d\d\d', regex=True))]).to_string()
  strDirectorio = re.sub(r'.*(C.\w\w.\w\w.)\d\d\d.*' , r'\1' ,StringDeTextoLimpioDeEsquema) #C.J6.75.
  # Crear intervalo de hojas que nos interesa
  listaHojas = list(range(0, pdfReader.getNumPages()))
  ## Crear un DataFrame con una columna que indique la "hoja" y otra que indique "esquema"
  dfHojaEsquema = pd.DataFrame()
  dfErroresDeLectura = pd.DataFrame()
  for i in listaHojas:
    page = pdfReader.getPage(i)
    page.extractText()
    lista = (str2lst(page.extractText()))
    df1DTexto = pd.Series(lista)
    StringDeTextoLimpioDeEsquema = (df1DTexto[(df1DTexto.str.contains(strDirectorio + '\d\d\d', regex=True))]) + 'JDIEZ'
    if (StringDeTextoLimpioDeEsquema.empty):
      dfErroresDeLectura = dfErroresDeLectura.append({'hoja sin contenido': i}, ignore_index=True)
    else:
      StringDeTextoLimpioDeEsquema = StringDeTextoLimpioDeEsquema.head(1).item()
      dfHojaEsquema = dfHojaEsquema.append({'hoja': (i), 'esquema': (re.sub(r'C.\w\w.\w\w.(\d\d\d)JDIEZ' , r'\1' ,StringDeTextoLimpioDeEsquema))}, ignore_index=True)
  return dfHojaEsquema, dfErroresDeLectura

# 3 - ENCONTRAR LISTA DE NÚMEROS A FILTRAR POR NO SER LÍNEAS
def GetListaStrNumerosFake(pdfReader):
  listaStrNumerosFake = []
  for pag in range(0,2):
    page = pdfReader.getPage(pag)
    page.extractText()
    lista = (str2lst(page.extractText()))
    df1DTexto = pd.Series(lista)
    dFConStringLineasSucio = (df1DTexto[(df1DTexto.str.contains('\d\d\d\d\d', regex=True))])
    # https://regex101.com/r/2kNe9f/1
    for i in (dFConStringLineasSucio):
      strLinea = re.sub(r'.*(\d\d\d\d\d)' , r'\1' ,i)
      listaStrNumerosFake.append(strLinea)
  listaStrNumerosFake = list(set(listaStrNumerosFake))
  return listaStrNumerosFake # Ej.: ['48708', '10269', '12439', '39490', '10269', '10319']

# 4 - CREAR SERIE QUE INDIQUE LAS LÍNEAS QUE HAY EN CADA HOJA
def GetSeriesLineasFiltradasPorHoja(pdfReader, listaStrNumerosFake, NumHoja):
  ## Crear un DataFrame con las hojas de las que queremos los IDs
  dfHojas = pd.DataFrame()
  page = pdfReader.getPage(NumHoja)
  aux = (str2lst(page.extractText()))
  dfHojas[NumHoja] = pd.Series(aux)
  ## Data cleaning (Lineas)
  # Todo lo que no sea un texto de 5 números lo sustituimos por un 0.
  dfHojas_cleaned = dfHojas.replace(to_replace ='[^\d\d\d\d\d]', value = 0, regex = True)
  # Los NaN también los sustituimos por 0s.
  dfHojas_cleaned = dfHojas_cleaned.fillna(0)
  # Convertimos los datos en numericos para poder filtrarlos aprovechando la condición numérica
  for j in (dfHojas_cleaned.columns):
    pd.to_numeric(dfHojas_cleaned[j])
    dfHojas_cleaned = dfHojas_cleaned.fillna(0)
    dfHojas_cleaned[j] = pd.to_numeric(dfHojas_cleaned[j], errors='coerce', downcast='integer')
    dfHojas_cleaned = dfHojas_cleaned.fillna(0)
    dfHojas_cleaned[j] = dfHojas_cleaned[j].astype(int)
  dfunified = dfHojas_cleaned.melt()['value']
  # Filtrar IDs por comparación numérica.
  dffiltered = dfunified[((dfunified >= 1000) & (dfunified < 100000)) | ( dfunified == 100 ) | ( dfunified == 500 ) | ( dfunified == 101 ) | ( dfunified == 501 )]
  # Eliminar duplicados.
  dfUnrepeated = dffiltered.drop_duplicates()
  # Ordenar IDs de menor a mayor
  dfSorted = dfUnrepeated.sort_values()
  # Eliminamos excepciones de los PDFs que están contenidos en los intervalos.
  for strNumeroFake in listaStrNumerosFake:
      # (5) Eliminamos excepciones de los PDFs que están contenidos en los intervalos.
      dfSorted = dfSorted[dfSorted != int(strNumeroFake)]
  dfSorted = dfSorted.drop_duplicates()
  return dfSorted

# 5 - CREAR DATAFRAME QUE INDIQUE LAS LÍNEAS QUE HAY EN CADA ESQUEMA CON REPETICIONES
def GetDfLineasPorEsquemaConRepeticiones(dfLineasFiltradasPorHoja, dfHojaEsquema):
  df1 = dfLineasFiltradasPorHoja.T
  df1 = df1.reset_index(drop=True)
  df2 = pd.DataFrame(dfHojaEsquema['esquema'])
  df2 = df2.reset_index(drop=True)
  dfLineasPorEsquema = pd.concat([df2, df1], axis=1)
  return dfLineasPorEsquema

# 6 - PROCESAR DATAFRAME QUE INDICA LAS LÍNEAS QUE HAY EN CADA ESQUEMA PARA QUE NO TENGA REPETICIONES
def GetDfLineasPorEsquemaSinRepetir(dfLineasPorEsquema):
  dfLineasPorEsquemaSinRepetir = pd.DataFrame()
  UltimoEsquemaGrabado = 0
  for esquema in dfLineasPorEsquema['esquema'].unique(): # str, ... 133 ...
    for row in range(0,(dfLineasPorEsquema[dfLineasPorEsquema.esquema == esquema]).shape[0]): # int, desde 0 hasta número de filas del df -> dfLineasPorEsquema[dfLineasPorEsquema.esquema == esquema]
      df = dfLineasPorEsquema[dfLineasPorEsquema.esquema == esquema]
      df = df.reset_index(drop=True)
      df1D = pd.DataFrame({esquema:df.loc[row, :]})
      if (UltimoEsquemaGrabado != esquema):
        UltimoEsquemaGrabado = esquema
        df1D = df1D.reset_index(drop=True)
        dfLineasPorEsquemaSinRepetir = dfLineasPorEsquemaSinRepetir.reset_index(drop=True)
        dfLineasPorEsquemaSinRepetir = pd.concat([dfLineasPorEsquemaSinRepetir, df1D], axis=1)
      else:
        df1D = df1D.reset_index(drop=True)
        dfLineasPorEsquemaSinRepetir = dfLineasPorEsquemaSinRepetir.reset_index(drop=True)
        dfLineasPorEsquemaSinRepetir[esquema] = pd.concat([dfLineasPorEsquemaSinRepetir[esquema], df1D[1:]], axis=0, ignore_index=True)
  return dfLineasPorEsquemaSinRepetir

# 7 - DESCARGAR DF DE LÍNEAS EN UN CSV
def DescargarCSVDfLineas(dfLineasPorEsquemaSinRepetir, strNombrePDF):
  dfLineasPorEsquemaSinRepetir = dfLineasPorEsquemaSinRepetir.drop(index=0)
  for i in dfLineasPorEsquemaSinRepetir.index:
    for j in dfLineasPorEsquemaSinRepetir.columns:
      if (math.isnan(dfLineasPorEsquemaSinRepetir[j][i]) != True): 
        dfLineasPorEsquemaSinRepetir[j][i] = int(dfLineasPorEsquemaSinRepetir[j][i])
  dfLineasPorEsquemaSinRepetir.to_csv('LineasPorEsquema_' + strNombrePDF + '.csv', index = True)
  files.download('LineasPorEsquema_' + strNombrePDF + '.csv')

# --> PROCESAMIENTO DE UN PDF POR CADA ITERACION
def procesoPorPDF(dbRutas, iterador):
  # PROCESO
  strNombrePDF = dbRutas['PDF name'][iterador]
  CodigoDrive = dbRutas['DRIVE code'][iterador]
  # (1) GetPDFObjFromDrive
  pdfReader = GetPDFObjFromDrive(CodigoDrive, strNombrePDF)
  # (2) GetDfEsquemaPorHoja
  dfHojaEsquema, dfErroresDeLectura = GetDfEsquemaPorHoja(pdfReader)
  # (3) GetListaStrNumerosFake
  listaStrNumerosFake = GetListaStrNumerosFake(pdfReader)
  # (4) GetSeriesLineasFiltradasPorHoja
  dfLineasFiltradasPorHoja = pd.DataFrame(GetSeriesLineasFiltradasPorHoja(pdfReader, listaStrNumerosFake, int(dfHojaEsquema['hoja'][0])))
  for NumHoja in ((dfHojaEsquema['hoja'][1:]).tolist()):
    df1 = dfLineasFiltradasPorHoja.reset_index(drop=True)
    df2 = GetSeriesLineasFiltradasPorHoja(pdfReader, listaStrNumerosFake, int(NumHoja)).reset_index(drop=True)
    df2 = df2.reset_index(drop=True)
    dfLineasFiltradasPorHoja = pd.concat([df1, df2], axis=1)
  # (5) Relacionamos en el mismo DataFrame los esquemas con las líneas de dicho esquema
  dfLineasPorEsquema = GetDfLineasPorEsquemaConRepeticiones(dfLineasFiltradasPorHoja, dfHojaEsquema)
  # (6) Procesamos el DF que contine las líneas por esquema con nombres de columna repetidas para que termine habiendo nombres de columnas 
  #     sin repetir con los datos de todas las columnas que anteriormente tenían el índice repetido.
  dfLineasPorEsquemaSinRepetir = GetDfLineasPorEsquemaSinRepetir(dfLineasPorEsquema)
  # (7) Descargar el csv del DataFrame resultante de líneas
  DescargarCSVDfLineas(dfLineasPorEsquemaSinRepetir, strNombrePDF)

##################################
## MAIN ##########################
##################################

# Ordenamos que se puedan mostrar muchos datos en el output del notebook
pd.options.display.max_rows = 999
# Datos concretos del PDF con el que vamos a trabajar.
dfDB_rutas_Drive=pd.read_csv('/content/drive/MyDrive/DIAGRAM SCRAPER/db.csv')
print('\nPDFs procesados:')
for iterador in range(0, dfDB_rutas_Drive.shape[0]):
  procesoPorPDF(dfDB_rutas_Drive, iterador)
  print(dfDB_rutas_Drive['PDF name'][iterador] + ' -> CSV descargable.')

PDFs procesados:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CK8_M -> CSV descargable.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CK8_TMC -> CSV descargable.
